In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, to_timestamp, hour, minute, year, month, day
from pyspark.sql.types import StringType, DoubleType, IntegerType, DateType, TimestampType

In [ ]:
spark = SparkSession.builder.master("local[*]") \
    .config("spark.executor.instances", "5") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY").getOrCreate()

In [ ]:
df = spark.read.option("Header", True).csv("spark-warehouse/landing/supermarket_sales.csv")

In [ ]:
df.columns

In [ ]:
df.printSchema()

In [ ]:
df.show(1)

In [ ]:
column_rename_and_cast = {
    'Invoice ID': ('invoice_id', StringType()),
    'Branch': ('branch', StringType()),
    'City': ('city', StringType()),
    'Customer type': ('customer_type', StringType()),
    'Gender': ('gender', StringType()),
    'Product line': ('product_line', StringType()),
    'Unit price': ('unit_price', DoubleType()),
    'Quantity': ('quantity', IntegerType()),
    'Tax 5%': ('tax_5_percent', DoubleType()),
    'Total': ('total', DoubleType()),
    'Date': ('date', StringType()), # Tratar data direto pode levar a problemas, melhor usar funções auxiliares
    'Time': ('time', StringType()), # Tratar timetamp direto pode levar a problemas, melhor usar funções auxiliares
    'Payment': ('payment', StringType()),
    'cogs': ('cogs', DoubleType()),
    'gross margin percentage': ('gross_margin_percentage', DoubleType()),
    'gross income': ('gross_income', DoubleType()),
    'Rating': ('rating', DoubleType())
}

In [ ]:
for original_name, (new_name, new_type) in column_rename_and_cast.items():
    df = df.withColumnRenamed(original_name, new_name) \
           .withColumn(new_name, col(new_name).cast(new_type))

In [ ]:
df.printSchema()

In [ ]:
df.explain()

In [ ]:
df.show()

In [ ]:
df = df.withColumn("year", year(to_date(col("date"), 'MM/dd/yyyy'))) \
        .withColumn("month", month(to_date(col("date"), 'MM/dd/yyyy'))) \
        .withColumn("day", day(to_date(col("date"), 'MM/dd/yyyy'))) \
        .withColumn("hour", hour(to_timestamp(col("time"), 'HH:mm'))) \
        .withColumn("minute", minute(to_timestamp(col("time"), 'HH:mm'))) \
        .drop(col("date")) \
        .drop(col("time"))

In [ ]:
df.printSchema()

In [ ]:
df.show()

In [ ]:
df.write.saveAsTable("supermarket_sales_bronze", format="parquet", mode="overwrite", partitionBy=["year", "month", "day", "hour", "minute"], path="bronze/supermarket_sales")